# 음악 추천

1.CSR matrix가 정상적으로 만들어졌다.

사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

2.MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.

사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

3.비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.

MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

cloud jupyter 를 열고 aiffel/recommendata_iu/ 디렉토리에 python3로 ipynb파일을 생성 해 주세요 jupyter notebook 환경에서 프로젝트 코드를 작성하고 제출하시면 됩니다.

이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
별점을 시청횟수로 해석해서 생각하겠습니다.
또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1) 데이터 준비와 전처리

In [1]:
import os
import pandas as pd
import numpy as np
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

- Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 평점과 무비 데이터프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [7]:
# 사용하지 않는 컬럼 제거
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


## 2) 데이터 분석

In [8]:
# 사용자 수와 영화의 수 구하기
num_watcher = ratings['user_id'].unique()
num_movies = ratings['movie_id'].unique()

print('number of watchers:',num_watcher)
print('number of movies:',num_movies)

number of watchers: [   1    2    3 ... 6038 6039 6040]
number of movies: [1193  661  914 ...  690 2909 1360]


In [9]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [10]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

## 3) 내가 선호하는 영화 추가

In [11]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('batman', regex=False)]

,movie_id,title,genre
151,153,Batman Forever (1995),Action|Adventure|Comedy|Crime
588,592,Batman (1989),Action|Adventure|Crime|Drama
1356,1377,Batman Returns (1992),Action|Adventure|Comedy|Crime
1522,1562,Batman & Robin (1997),Action|Adventure|Crime
3144,3213,Batman: Mask of the Phantasm (1993),Animation|Children's


In [12]:
# 내가 좋아하는 영화 5편의 id와 title 리스트 만들기
my_favorite_id = [153, 592, 1377 , 1562, 3213]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# '101010'라는 user_id가 위 5편의 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['101010']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
# 처음에 id이름을 Hyunji라고 설정해줬으나 base10 error가 떠서 101010으로 설정했다.
my_movielist

,user_id,movie_id,count,title
0,101010,153,5,Batman Forever (1995)
1,101010,592,5,Batman (1989)
2,101010,1377,5,Batman Returns (1992)
3,101010,1562,5,Batman & Robin (1997)
4,101010,3213,5,Batman: Mask of the Phantasm (1993)


In [13]:
if not ratings.isin({'user_id':['101010']})['user_id'].any():  # user_id에 '101010'라는 데이터가 없다면
    ratings = ratings.append(my_movielist, ignore_index=True)   # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(8)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,101010,153,5,Batman Forever (1995)
836479,101010,592,5,Batman (1989)
836480,101010,1377,5,Batman Returns (1992)
836481,101010,1562,5,Batman & Robin (1997)
836482,101010,3213,5,Batman: Mask of the Phantasm (1993)


## 4) CSR matrix

In [14]:
num_watcher = ratings['user_id'].unique()
num_movies = ratings['title'].unique()


In [15]:
# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v:k for k,v in enumerate(num_watcher)}
movie_to_idx = {v:k for k,v in enumerate(num_movies)}

# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column 인덱싱 성공!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id 인덱싱 실패')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column 인덱싱 성공!!')
    ratings['title'] = temp_movie_data
else:
    print('title column 인덱싱 실패')

ratings

user_id column 인덱싱 성공!!
title column 인덱싱 성공!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,153,5,515
836479,6039,592,5,527
836480,6039,1377,5,587
836481,6039,1562,5,1943


In [16]:
num_watcher = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()

# CSR Matrix 생성
csr_data = csr_matrix((ratings['count'], (ratings.movie_id, ratings.title)), shape= (num_watcher, num_movies))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 3628 stored elements in Compressed Sparse Row format>

## 5) AlternatingLeastSquares 모델 훈련

In [17]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

- AlternatingLeastSquares 클래스의 __init__ 파라미터
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

## 6) 선호도 파악

In [23]:
ratings

,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,153,5,515
836479,6039,592,5,527
836480,6039,1377,5,587
836481,6039,1562,5,1943


In [29]:
# 훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
hyunji, SS = user_to_idx['101010'], movie_to_idx['Matrix, The (1999)']
hyunji_vector, SS_vector = als_model.user_factors[hyunji], als_model.item_factors[SS]

hyunji_vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [19]:
SS_vector

array([ 2.1117565e-03, -1.1053169e-03, -2.7744421e-03, -2.7547128e-04,
        4.9718432e-03,  1.2551512e-03,  4.3985620e-03, -2.0671363e-03,
        7.0989039e-03, -1.9426034e-03,  3.8332893e-03, -1.4812633e-04,
        4.5305337e-03, -1.1767657e-03, -1.0648480e-03,  2.5188804e-03,
        7.1107792e-03, -1.8562822e-03,  1.9754225e-04,  1.1284841e-03,
        1.6573409e-03,  5.0451392e-03, -1.5094432e-03,  6.1626742e-03,
        6.9085835e-03, -3.6038822e-03,  3.6353532e-03, -1.1092757e-03,
       -6.3855527e-04,  2.9775244e-03,  4.4799151e-04,  2.3913388e-03,
       -4.9853069e-04, -6.2963320e-04,  9.5227401e-04, -1.4356718e-03,
       -1.6610265e-03,  3.0603430e-03, -3.9789607e-03,  4.0255557e-03,
        1.6667023e-04, -2.6395973e-03, -1.6967339e-03, -2.1389413e-03,
        3.4375404e-04,  2.5560430e-03, -2.0628162e-03, -2.3819145e-03,
       -2.2165149e-03,  4.9138539e-03,  3.0562789e-03, -2.9368200e-03,
        4.1527338e-03,  1.9064343e-03, -9.3373097e-04,  3.2544890e-03,
      

In [20]:
# 나와 매트릭스의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(hyunji_vector, SS_vector)

0.0

- 0이 뜨면 안되는데 0이 뜨는 이유를 잘 파악하지 못했습니다. 

## 7) 좋아하는 영화와 비슷한 영화 추천

In [30]:
favorite_movie = 'Matrix, The (1999)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple

[(124, 1.0),
 (2862, 0.50531286),
 (2267, 0.5002354),
 (3102, 0.4941743),
 (3468, 0.4910983),
 (3503, 0.4887739),
 (3256, 0.48714948),
 (3365, 0.4840936),
 (3011, 0.4831578),
 (3119, 0.48209503),
 (3189, 0.48192286),
 (3161, 0.48190424),
 (3280, 0.4800788),
 (3421, 0.47845954),
 (3328, 0.478002)]

In [31]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Matrix, The (1999)',
 'Substance of Fire, The (1996)',
 'Designated Mourner, The (1997)',
 'Bad Moon (1996)',
 'Montana (1998)',
 'Santitos (1997)',
 'Fear, The (1995)',
 'Two or Three Things I Know About Her (1966)',
 'Kids of the Round Table (1995)',
 'Ogre, The (Der Unhold) (1996)',
 'Temptress Moon (Feng Yue) (1996)',
 'Alarmist, The (1997)',
 "I Can't Sleep (J'ai pas sommeil) (1994)",
 'Taffin (1988)',
 'Bat, The (1959)']